# <center>Capstone Project - The Battle of Neighbourhoods: Welsh Towns Review</center>
## <center>Part 3 - Methodology</center>
### Applied Data Science Capstone by IBM
### Part of our IBM Data Science Professional Certificate

***

## Table of contents
* [The Problem](#section1)
* [Data Analysis](#section2)
    * [A. Secondary Schools Data](#section3)
    * [B. Average Property Prices per County in Wales](#section4)
    * [C. Correlation between the school rating and average property price](#section5)
    * [D. Clustering of the towns](section6)
* [Presentation of Results](#section7)


## The Problem <a name="section1"></a>

A couple with young children is looking for a safe and quiet place to live. For their children they want a good state school and for the family a small, but vibrant town. They would like either to settle in that town or very close. They are flexible in regards to the location because they both work from home with only occasional business travels to a city. But where to start? Where are the good schools and which towns could be nice to live in?

## Data Analysis<a name="section2"></a>

Load required libraries

In [ ]:
# for data analysis
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np

# for charting:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# for presentation on a map: 
import folium


#### A. Secondary Schools Data<a name="section3"></a>

In the previous workbooks we downloaded and cleaned the data: <br>
1. Download list of schools in Wales including their 2019 rating:<br>
https://gov.wales/sites/default/files/publications/2020-02/national-school-categorisation-system-support-categories-2019-v2.xlsx 
It is a large file with complex headings and multiple tabs. The most efficient way of loading the date is by pre-processing it in Ms Excel. Primary and special schools have been removed as well as the empty cells. Finally the headings were simplified.
2.  To present the schools on a map, their addresses are obtained from the Welsh Governent’s website: <br>
https://gov.wales/sites/default/files/publications/2021-02/address-list-schools-wales.ods. <br>
The two tables were joined and the size reduced to few most important columns only: `['School_code', 'School_name', 'Local_authority', 'Rating', 'Postcode']`.
3. The data set was geolocated using 'Geocoder' and results saved as: `schools_geo.csv`

Load the dataset:

In [ ]:
schools = pd.read_csv('schools_geo.csv')
schools.info()
schools.head()

Separate the yellow and green schols to display them in different colours

In [ ]:
# Create a df with yellow rated schools 'Green/Gwyrdd': 
schools_g = schools[schools.Rating == 'Green/Gwyrdd']
schools_g.info()
schools_g.head()

In [ ]:
# Create a df with yellow rated schools 'Yellow/Melyn': 
schools_y = schools[schools.Rating == 'Yellow/Melyn']
schools_y.info()
schools_y.head()

#### B. Average Property Prices per County in Wales<a name="section4"></a>

Load data prepared in the previous notebooks:

In [ ]:
# Average property prices per county downloaded from uk.gov website:
counties = pd.read_csv('Prices_Wales.csv') # two of the county names required a slight edit to match the other schools table.

counties.info()
counties.head()

In [ ]:
counties.describe()

In [ ]:
# Load the data-set with all schools rated, drop columns that are not required,
df1 = pd.read_csv('schools_rated.csv')
df1 = df1.drop(['Consortium', 'LA Code', 'Sector', 'Governance - see notes', 'WM Code', 'Welsh Medium Type - see notes', 'School Type', 'Religious Character', 'Address 1', 'Address 2', 
                'Address 3', 'Address 4', 'Postcode', 'Phone Number', 'Pupils - see notes'], axis = 1)

In [ ]:
# Create a pivot counting the number of schools per 'Local_authority':

df2 = pd.pivot_table(df1, values = 'School_code', index = ['Local_authority'], aggfunc = 'count')
df2.rename(columns = {'School_code':'No_Schools'}, inplace = True)

# Create a pivot counting the number of schools with each rating per 'Local_authority':
df3 = pd.pivot_table(df1, values = 'School_code', index = ['Local_authority'], columns = 'Rating', aggfunc = 'count')
df3.fillna(0, inplace = True)

# Merge the two data frames:
df4 = pd.merge(df2, df3, on = 'Local_authority')

In [ ]:
# Add columns with proportion of schools in each category:
df4['Green%'] = df4['Green/Gwyrdd'] / df4['No_Schools']
df4['Yellow%'] = df4['Yellow/Melyn'] / df4['No_Schools']
df4['Amber%'] = df4['Amber/Oren'] / df4['No_Schools']
df4['Red%'] = df4['Red/Coch'] / df4['No_Schools']

# Reset index
#df4.reset_index(drop = False)
df4.reset_index(inplace = True)
df4

#### C. Correlation between the school rating and average property price<a name="section5"></a>
Now we can to see whether there is correlation between the school rating and average property prices:

In [ ]:
# Remove Welsh county name from 'Local_authority' to enable data match
df4['Local_authority'].replace(to_replace=r' / .*', value='', regex=True, inplace=True)

# Rename 'County' column to enable matching
counties.rename(columns = {'County':'Local_authority'}, inplace = True)

# add average property price to the df4:
df5 = pd.merge(df4, counties, on = 'Local_authority')

df5

In [ ]:
df5 = df5.drop(['No_Schools', 'Amber/Oren', 'Green/Gwyrdd', 'Red/Coch', 'Yellow/Melyn'], axis=1)

In [ ]:
df5.corr()

In [ ]:
#plot a Seaborn heatmap
fig, ax = plt.subplots(figsize=(20,10)) # size of a single point/field in inches
# Draw a heatmap with the numeric values in each cell
sns.heatmap(df5.corr(), annot=True, fmt=".0%", linewidths=.5, ax=ax)

To picture the results clearly two new columns were added to the data frame. One column converted the price values into a 0 – 1 range showing each county’s average property price against the minimum and maximum difference, where 1 is the cheapest and 0 is the most expensive county. The same was done with the proportion of green schools within each county’s school number, but here 1 means the highest proportion of schools ranked ‘green’ and ‘0’ the lowest.
In this way, xx shows the best value for money in terms of excellent schools when the county has plotted the closet to the top right: both values going towards ‘1’. 

In [ ]:
df5['Price_Index'] = (max(df5['Ave_price_2021']) - df5['Ave_price_2021']) / (max(df5['Ave_price_2021'])-min(df5['Ave_price_2021']))
df5['School_Index'] = (df5['Green%']-min(df5['Green%'])) / (max(df5['Green%'])-min(df5['Green%']))

In [ ]:
df6 = df5.drop(['Yellow%', 'Amber%', 'Red%'], axis=1)
df6

In [ ]:
sns.scatterplot(data = df6, x = 'Price_Index', y = 'School_Index', size = 'School_Index', legend = 'auto')

In [ ]:
# Data:
x = df6['Price_Index']
y = df6['School_Index']
types = df6['Local_authority'].values

# Marker size (s) and colour (c):
price = df6['Price_Index'] * 500
school = df6['School_Index'] * 500

fig, ax = plt.subplots(figsize = (10, 10))
ax.scatter(x, y, c=price, s=school, alpha=1)

ax.set_xlabel(r'Price Index', fontsize=15)
ax.set_ylabel(r'School Index', fontsize=15)
ax.set_title('Ranking of Counties: property price and school ranking')

# Label data points:
for i, txt in enumerate(types):
    ax.annotate(txt, (x[i], y[i]), xytext=(10,10), textcoords='offset points')
    plt.scatter(x, y, marker='None', color='black')

#ax.legend()
ax.grid(True) 

plt.show()

#### D. Clustering of the towns <a name="section6"></a>

We have already downloaded the venue data for all 132 towns / localities in Wales using Four Square API.

In [ ]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [ ]:
df7 = pd.read_csv('venues.csv')
df7.rename(columns = {'Neighbourhood':'Town', 'Neighbourhood Latitude':'T_Lati', 'Neighbourhood Longitude':'T_Long'}, inplace = True)
df7.info()
df7.head()

Check how many venues per town the search returned:

In [ ]:
print(df7.groupby('Town').count())

Check the unique venue categories that Four Square returned and their number:

In [ ]:
df7['Venue Category'].value_counts()

In [ ]:
print("there are ", df7['Venue Category'].nunique(), " unique venue types in the selected towns")

#### Prepare data for clustering

convert the dataframe to have the unique venues as colums

In [ ]:
# venues as columns:
wales_onehot = pd.get_dummies(df7[['Venue Category']], prefix="", prefix_sep="")

# add town column back to dataframe
wales_onehot['Town'] = df7['Town'] 

# move neighbourhood column to the first column
fixed_columns = [wales_onehot.columns[-1]] + list(wales_onehot.columns[:-1])
wales_onehot = wales_onehot[fixed_columns]

wales_onehot.info()
wales_onehot.head()

Group rows by town and by taking the mean of the frequency of occurrence of each category normalise the data set.

In [ ]:
wales_grouped = wales_onehot.groupby('Town').mean().reset_index()
print('Size of the new table: ', wales_grouped.shape)
wales_grouped.head()

In [ ]:
wales_grouped.describe()

Print each neighbourhood along with the top 10 most common venues:

In [ ]:
num_top_venues = 3

for town in wales_grouped['Town']:
    print("----"+town+"----")
    temp = wales_grouped[wales_grouped['Town'] == town].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 3})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

To put the data in a dataframe we can first sort the venues:

In [ ]:
# define the sorting function:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create the new dataframe and display the top 10 venues for each neighbourhood:

In [ ]:
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Town']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
towns_venues_sorted = pd.DataFrame(columns=columns)
towns_venues_sorted['Town'] = wales_grouped['Town']

for ind in np.arange(wales_grouped.shape[0]):
    towns_venues_sorted.iloc[ind, 1:] = return_most_common_venues(wales_grouped.iloc[ind, :], num_top_venues)

print('Size of the new table: ', towns_venues_sorted.shape)
towns_venues_sorted.head()

In [ ]:
# save the table as csv to append in the report:
# towns_venues_sorted.to_csv(r'towns_venues_sorted.csv', index = False)

Add normalised town population

In [ ]:
df8 = pd.read_csv('towns_geo.csv')

In [ ]:
df8 = df8.drop(['Latitude', 'Longitude'], axis = 1)

In [ ]:
df8['Pop_Index'] = (df8['Population']-min(df8['Population'])) / (max(df8['Population'])-min(df8['Population']))

In [ ]:
df8 = df8.drop(['Population'], axis = 1)

In [ ]:
df9 = pd.merge(wales_grouped, df8, on = 'Town', how = 'left')

In [ ]:
df9

#### Cluster the neigbourhoods using K-Means model

In [ ]:
# import k-means from clustering stage
from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [ ]:
# set number of clusters
kclusters = 6

wales_grouped_clustering = df9.drop('Town', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(wales_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

In [ ]:
# create a new dataframe that includes the cluster as well as the top 3 venues for each neighbourhood

# add clustering labels
towns_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

wales_merged = pd.read_csv('towns_geo.csv')

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighbourhood
wales_merged = wales_merged.join(towns_venues_sorted.set_index('Town'), on='Town')

wales_merged.head()

In [ ]:
wales_merged.sort_values(by = ['Cluster Labels'], inplace = True)
wales_merged

## Presentation of Results <a name="section7"></a>

In [ ]:
# Load the towns data:
towns = pd.read_csv('towns_geo.csv')
# Create a centre point coordinations for Wales using the mean lat-lon from the dataframe:
lat = towns['Latitude'].mean()
lon = towns['Longitude'].mean()
print('Wales mean coordinates /lat - lon/: ',lat,' , ',lon)
# Load the county boundaries downloaded from the gov.uk website:
geo_data = r'Counties_and_Unitary_Authorities_(December_2016)_Boundaries.geojson'

### Blank Map

In [ ]:
# Setup a blank map of Wales:
map_w = folium.Map(location=[lat, lon], zoom_start=7, tiles='cartodbpositron')

### Counties

In [ ]:
# Add the counties Choropleth coloured by the average price:

choropleth = folium.Choropleth(
    geo_data,
    data=counties, # my dataset with prices
    columns=['Local_authority', 'Ave_price_2021'], # 'County' is here for matching the geojson 'ctyua16nm', 'Ave_price_2021' is the column that changes the color of zipcode areas
    key_on='feature.properties.ctyua16nm', # this path contains counties in str type and should match with our 'County column
    fill_color='BuPu',
    fill_opacity=0.4,
    line_color= 'None',
    legend_name='Average Property Price, Wales 2021',
    nan_fill_color = 'None'
).add_to(map_w)

# add labels indicating the name of the county
style_function = "font-size: 10px; font-weight: normal"
choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['ctyua16nm'], style=style_function, labels=False)
)

### Schools

In [ ]:
# add schools to map
# Green:
for lat_s, lng_s, school, rating in zip(schools_g['Latitude'], schools_g['Longitude'], schools_g['School_name'], schools_g['Rating']):
    label = '{}'.format(str(school) + ' support category: ' + str(rating))
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat_s, lng_s],
        radius=4,
        stroke=True,
        color='black',
        weight=2,
        fill=True,
        fill_color='green',
        fill_opacity=1,
        popup=label,
        parse_html=False).add_to(map_w)  
    
# Yellow:
for lat_s, lng_s, school, rating in zip(schools_y['Latitude'], schools_y['Longitude'], schools_y['School_name'], schools_y['Rating']):
    label = '{}'.format(str(school) + ' support category: ' + str(rating))
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat_s, lng_s],
        radius=4,
        stroke=True,
        color='black',
        weight=2,
        fill=True,
        fill_color='yellow',
        fill_opacity=1,
        popup=label,
        parse_html=False).add_to(map_w)  


### Towns - Clustered

In [ ]:
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to map
markers_colors = []
for lat, lng, poi, cluster in zip(wales_merged['Latitude'], wales_merged['Longitude'], wales_merged['Town'], wales_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        stroke=False,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7,
        parse_html=False).add_to(map_w)  
    

In [ ]:
# add tile layers to the map:
tiles = ['stamenwatercolor', 'cartodbpositron', 'openstreetmap', 'stamenterrain']
for tile in tiles:
    folium.TileLayer(tile).add_to(map_w)

# create a layer control:
folium.LayerControl().add_to(map_w)

# plot the map:
map_w

The above interactive map concludes the project as browsable results